In [3]:
import pandas as pd
import os

In [18]:
path="/home/data/meme_challenge_mod_data"
title="simple_"

image_features=pd.read_csv(os.path.join(path,title+"image_features.csv"),sep=",",index_col=0)
text_features=pd.read_csv(os.path.join(path,title+"text_features.csv"),sep=",",index_col=0)
desc_features=pd.read_csv(os.path.join(path,title+"text_features.csv"),sep=",",index_col=0)
''' 
results.to_csv()

results=pd.DataFrame(desc)
results.to_csv(os.path.join(path,title+"description.csv"),sep=",")

results=pd.DataFrame(img_features)
results.to_csv(os.path.join(path,title+"image_features.csv"),sep=",")
'''

' \nresults.to_csv()\n\nresults=pd.DataFrame(desc)\nresults.to_csv(os.path.join(path,title+"description.csv"),sep=",")\n\nresults=pd.DataFrame(img_features)\nresults.to_csv(os.path.join(path,title+"image_features.csv"),sep=",")\n'

,0,1,2,3,4,5,6,7,8,9,...,758,759,760,761,762,763,764,765,766,767
0,0.673370,0.421808,0.883554,0.052322,0.107380,1.314931,1.468652,0.926652,0.339213,-0.120326,...,-0.516278,0.182493,0.930543,-0.993845,-0.711212,-0.274342,-0.166725,0.283385,-1.071380,0.281744
1,0.231843,0.623879,1.265641,0.303393,-0.335262,0.212764,1.930729,0.480074,0.649091,-0.105371,...,-0.194136,-0.325853,-0.149727,-1.848536,-0.272475,-0.493032,0.108963,-0.104366,-0.320971,0.194518
2,-0.167888,-0.345920,1.369281,0.215230,-0.186173,0.515326,0.116858,-0.933484,0.002899,-0.083381,...,-0.293832,-1.082375,-0.380994,-0.298385,0.150651,0.122679,0.143505,0.252890,0.004613,0.256221
3,-0.279274,1.449150,-0.294184,-0.071279,-0.501457,-0.721508,1.070244,0.586749,0.116689,-0.350241,...,-0.959610,-0.305082,-0.716637,-1.789879,0.039059,-0.674058,0.220202,0.635446,0.634702,0.695357
4,-0.582884,1.646930,-0.427831,-0.345590,-0.310103,-1.427373,-0.050149,0.686717,0.361579,-0.113115,...,-1.289123,-1.156750,-0.862771,-0.768398,0.145734,-1.016364,0.304564,0.489634,0.586980,0.462873
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
94,0.139328,0.010928,0.906224,0.135862,0.177266,-0.190389,-0.649339,0.544706,0.332107,-0.559273,...,0.840008,-0.341225,-0.636761,-0.305199,-0.465476,-0.651017,0.397587,-0.030657,-0.377774,0.054580
95,-0.292066,0.688459,0.921568,0.239495,-0.722636,-1.112103,0.607104,0.324240,0.214675,-0.208295,...,-0.420754,-0.913447,-0.617840,-1.734759,-0.028196,-0.289787,-0.244782,-0.137169,0.037343,-0.061367
96,0.105471,0.503345,1.012848,0.405386,-0.161673,0.762337,0.697213,-0.087105,-0.231605,-0.280838,...,1.086555,0.082666,0.436932,-0.869155,-0.045337,-0.175238,-0.867009,-0.083973,-0.237392,-0.590160
97,0.245425,0.315950,1.646027,-0.059400,-0.651954,0.165287,1.352939,0.793449,-0.152364,-0.111468,...,-0.207726,-0.560793,-0.074031,-1.323362,-0.597310,-0.974062,-0.121984,0.407427,-0.526450,0.282667
